In [94]:
from openclean.data.load import dataset
from openclean.pipeline import stream
import pandas as pd

pd.set_option('display.max_rows', None)

data_dir = '../project_data/'

data_list = [
    'data-cityofnewyork-us.hy4q-igkk.csv', 
    'data-cityofnewyork-us.aiww-p3af.csv', 
    'data-cityofnewyork-us.3rfa-3xsf.csv', 
    'data-cityofnewyork-us.acdt-2zt9.csv', 
    'data-cityofnewyork-us.cwy2-px8b.csv', 
    'data-cityofnewyork-us.xrwg-eczf.csv'
]

data_column = [
    'Agency Name', 
    'Agency Name', 
    'Agency Name', 
    'Agency Name', 
    'Agency Name',
    'Agency Name'
]

In [77]:
# our original strategy:
# 1. padding missing value with value 'UNKNOWN'
# 2. using knn cluster to check the spelling error
from openclean.cluster.knn import knn_clusters, knn_collision_clusters
from openclean.function.similarity.base import SimilarityConstraint
from openclean.function.similarity.text import LevenshteinDistance
from openclean.function.value.threshold import GreaterThan

from openclean.function.value.null import is_empty
from openclean.operator.transform.update import update


def perform_knn_cluster(ds_full, column, using_collision=True, minsize=2, t=0.9):
    values = ds_full.select(column).distinct()
    clusters = knn_clusters(values=values, sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(t)), minsize=minsize) \
        if using_collision else knn_collision_clusters(values=values, sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(t)), minsize=minsize)
    print('{} clusters of size {} or greater'.format(len(clusters), minsize))
    clusters.sort(key=lambda c: len(c), reverse=True)
    for i, cluster in enumerate(clusters):
        print_cluster(i + 1, cluster)
        
def print_cluster(cnumber, cluster):
    print('Cluster {} (of size {})\n'.format(cnumber, len(cluster)))
    for val, count in cluster.items():
        print('{} ({})'.format(val, count))
    print('\nSuggested value: {}\n\n'.format(cluster.suggestion()))

In [78]:
# load all the dataset at once then performing data cleaning to create reference data
# Then we profiling the dataset

Agency_df = pd.DataFrame(columns=['Agency Name'])

for i in range(len(data_list)):
    datafile = data_dir + data_list[i]
    print("Load data: ", datafile)
    ds = dataset(datafile, encoding='utf-8')
   
    Agency = ds[['Agency Name']]
    Agency_df = Agency_df.append(Agency)
    
print(Agency_df.value_counts())

Load data:  ../project_data/data-cityofnewyork-us.hy4q-igkk.csv
Load data:  ../project_data/data-cityofnewyork-us.aiww-p3af.csv
Load data:  ../project_data/data-cityofnewyork-us.3rfa-3xsf.csv
Load data:  ../project_data/data-cityofnewyork-us.acdt-2zt9.csv
Load data:  ../project_data/data-cityofnewyork-us.cwy2-px8b.csv
Load data:  ../project_data/data-cityofnewyork-us.xrwg-eczf.csv
Agency Name                                                                                
Department of Housing Preservation and Development                                             2015506
Department of Transportation                                                                    965161
Department of Environmental Protection                                                          642485
New York City Police Department                                                                 548551
Department of Buildings                                                                         381210
Departmen

In [79]:
# Generate our reference data
Agency_df.to_csv('../reference_data/agency_rows_all.csv', index=None)

In [80]:
# Load the intermediate dataset as stream format
ds_full = stream('../reference_data/agency_rows_all.csv', encoding='utf-8')

In [81]:
# First we perform knn cluster
perform_knn_cluster(ds_full, 'Agency Name')

66 clusters of size 2 or greater
Cluster 1 (of size 7)

School - PS 255 (3)
School - PS 256 (2)
School - PS 214 (2)
School - PS 234 (1)
School - PS 224 (1)
School - PS 24 (1)
School - PS 254 (1)

Suggested value: School - PS 255


Cluster 2 (of size 7)

School - PS 245 (1)
School - PS 22 (3)
School - PS 214 (2)
School - PS 234 (1)
School - PS 224 (1)
School - PS 254 (1)
School - PS 24 (1)

Suggested value: School - PS 22


Cluster 3 (of size 7)

School - PS 231 (5)
School - PS 214 (2)
School - PS 224 (1)
School - PS 24 (1)
School - PS 254 (1)
School - PS 134 (1)
School - PS 234 (1)

Suggested value: School - PS 231


Cluster 4 (of size 6)

School - PS 22 (3)
School - PS 214 (2)
School - PS 234 (1)
School - PS 24 (1)
School - PS 254 (1)
School - PS 224 (1)

Suggested value: School - PS 22


Cluster 5 (of size 6)

School - PS 211 (1)
School - PS 234 (1)
School - PS 224 (1)
School - PS 24 (1)
School - PS 254 (1)
School - PS 214 (2)

Suggested value: School - PS 214


Cluster 6 (of size 6)

In [82]:
# After inspecting the KNN Clustering result, we noticed that there was not obvious typo error in this cloumn detected by the KNN
# Next we transform the data to DataFrame and then check missing values in the column

df_overall = ds_full.to_df()
print(df_overall.columns)

Index(['Agency Name'], dtype='object')


In [83]:
# Inspect rows with missing values 
df_missing = df_overall[df_overall['Agency Name'].isnull()]
print(df_missing.count())

Agency Name    0
dtype: int64


It was noticed that no values in these dataset was missing
hence we assume that the column 'Agency Name' was correct in these dataset collection
Next we refine our strategy
We noticed that there was a functional dependency between column 'Agency' and 'Agency Name'
'Agency' was the abbreviation of the department the agency belongs to
Hence we extract the dataset with both column 'Agency' and 'Agency Name'

In [84]:
# We listed all dataset with both columns
ref_file_list = [
    'data-cityofnewyork-us.3rfa-3xsf.csv', 
    'data-cityofnewyork-us.aiww-p3af.csv', 
    'data-cityofnewyork-us.cwy2-px8b.csv', 
    'data-cityofnewyork-us.hy4q-igkk.csv', 
    'data-cityofnewyork-us.xrwg-eczf.csv'
]

columns = ['Agency', 'Agency Name']
df_agency_map = pd.DataFrame(columns=columns)

In [85]:
# And load the corresponding dataset, saved them to a new reference dataframe

for file_name in ref_file_list:
    datafile = data_dir + file_name
    df = dataset(datafile, encoding='utf-8')
    agency_name = df[['Agency', 'Agency Name']]
    df_agency_map = df_agency_map.append(agency_name)
    
print(df_agency_map.count())

Agency         5471311
Agency Name    5471311
dtype: int64


In [86]:
# Profile the column to see whether there was a linking error between these two columns
print(df_agency_map.value_counts())

Agency      Agency Name                                                                                
HPD         Department of Housing Preservation and Development                                             2015506
DOT         Department of Transportation                                                                    965161
DEP         Department of Environmental Protection                                                          642485
NYPD        New York City Police Department                                                                 548551
DOB         Department of Buildings                                                                         381210
DPR         Department of Parks and Recreation                                                              165511
DOHMH       Department of Health and Mental Hygiene                                                         134952
DCA         Department of Consumer Affairs                                                 

After inspected into the mapping result, we noticed that some Agency Name column has the same value as Agency.
Such as DCA and TLC.
We assume them as problem data

In [87]:
# Since we noticed there was correct mapping values in the dataset
# We dropped them and generate a reference data
# First we located them
df_dup = df_agency_map[df_agency_map['Agency'] == df_agency_map['Agency Name']]
print(df_dup.count())

Agency         190
Agency Name    190
dtype: int64


In [88]:
# We drop these problem dataset
df_agency_clean = df_agency_map.drop(df_dup.index)

print(df_agency_clean.value_counts())

Agency      Agency Name                                                                                
HPD         Department of Housing Preservation and Development                                             2015492
DOT         Department of Transportation                                                                    964936
DEP         Department of Environmental Protection                                                          642430
NYPD        New York City Police Department                                                                 548487
DOB         Department of Buildings                                                                         381193
DPR         Department of Parks and Recreation                                                              165489
DOHMH       Department of Health and Mental Hygiene                                                         134948
DCA         Department of Consumer Affairs                                                 

In [89]:
# We save them as reference data

group = df_agency_clean.value_counts().reset_index()
group.columns = ['Agency', 'Agency Name', 'count']

df_ref = group[['Agency', 'Agency Name']]
df_ref.to_csv('../reference_data/agency_reference_data.csv', index=None)

In [90]:
# Then we use this reference data to clean all the values

agency_abb = dataset('../reference_data/agency_reference_data.csv')
agency_abb = dict(zip(agency_abb['Agency'],agency_abb['Agency Name']))

In [92]:
# Apply our new reference to the overall datasets

df_dup = df_agency_map[df_agency_map['Agency'] == df_agency_map['Agency Name']]
df_agency_map = update(df_agency_map, columns='Agency Name', func=lambda x: agency_abb[str(x)] if str(x) in agency_abb else x)
print(df_agency_map.value_counts())

Agency      Agency Name                                                                                
HPD         Department of Housing Preservation and Development                                             2015506
DOT         Department of Transportation                                                                    965161
DEP         Department of Environmental Protection                                                          642485
NYPD        New York City Police Department                                                                 548551
DOB         Department of Buildings                                                                         381210
DPR         Department of Parks and Recreation                                                              165516
DOHMH       Department of Health and Mental Hygiene                                                         134957
DCA         Department of Consumer Affairs                                                 

In [103]:
# Now we write our new strategy function

def improved_data_clean_agency(ds_full, ref_map):
    ds_full = update(ds_full, columns='Agency Name', func=lambda x: 'UNSPECIFIED' if is_empty(x) else x)
    df_full = update(ds_full, columns='Agency Name', func=lambda x: ref_map[str(x)] if str(x) in ref_map else x)
    ds_full = update(ds_full, columns='Agency Name', func=str.upper)
    return ds_full

Then we extend our new strategy to all the dataset

## data-cityofnewyork-us.hy4q-igkk.csv

In [99]:
datafile = data_dir + data_list[0]
df = dataset(datafile, encoding='utf-8')
df = df[['Agency Name']]
print(df.value_counts())

Agency Name                                                                              
Department of Housing Preservation and Development                                           654170
Department of Transportation                                                                 312300
Department of Environmental Protection                                                       228001
New York City Police Department                                                              190219
Department of Buildings                                                                      117360
Department of Parks and Recreation                                                            45960
Department of Health and Mental Hygiene                                                       43090
Department of Consumer Affairs                                                                29579
Taxi and Limousine Commission                                                                 21172
BCC - Broo

In [104]:
df = improved_data_clean_agency(df, agency_abb)
print(df.value_counts())

Agency Name                                                                              
DEPARTMENT OF HOUSING PRESERVATION AND DEVELOPMENT                                           654170
DEPARTMENT OF TRANSPORTATION                                                                 312300
DEPARTMENT OF ENVIRONMENTAL PROTECTION                                                       228001
NEW YORK CITY POLICE DEPARTMENT                                                              190219
DEPARTMENT OF BUILDINGS                                                                      117360
DEPARTMENT OF PARKS AND RECREATION                                                            45960
DEPARTMENT OF HEALTH AND MENTAL HYGIENE                                                       43090
DEPARTMENT OF CONSUMER AFFAIRS                                                                29579
TAXI AND LIMOUSINE COMMISSION                                                                 21172
BCC - BROO

## data-cityofnewyork-us.aiww-p3af.csv

In [105]:
datafile = data_dir + data_list[1]
df = dataset(datafile, encoding='utf-8')
df = df[['Agency Name']]
print(df.value_counts())

Agency Name                                                                   
Department of Housing Preservation and Development                                689717
Department of Transportation                                                      319280
Department of Environmental Protection                                            246571
New York City Police Department                                                   185521
Department of Buildings                                                           134357
Department of Parks and Recreation                                                 52388
Department of Health and Mental Hygiene                                            44945
Department of Consumer Affairs                                                     28813
Taxi and Limousine Commission                                                      23273
BCC - Brooklyn South                                                               17913
BCC - Brooklyn North           

In [106]:
df = improved_data_clean_agency(df, agency_abb)
print(df.value_counts())

Agency Name                                                                   
DEPARTMENT OF HOUSING PRESERVATION AND DEVELOPMENT                                689717
DEPARTMENT OF TRANSPORTATION                                                      319280
DEPARTMENT OF ENVIRONMENTAL PROTECTION                                            246571
NEW YORK CITY POLICE DEPARTMENT                                                   185521
DEPARTMENT OF BUILDINGS                                                           134357
DEPARTMENT OF PARKS AND RECREATION                                                 52388
DEPARTMENT OF HEALTH AND MENTAL HYGIENE                                            44945
DEPARTMENT OF CONSUMER AFFAIRS                                                     28813
TAXI AND LIMOUSINE COMMISSION                                                      23273
BCC - BROOKLYN SOUTH                                                               17913
BCC - BROOKLYN NORTH           

## data-cityofnewyork-us.3rfa-3xsf.csv

In [107]:
datafile = data_dir + data_list[2]
df = dataset(datafile, encoding='utf-8')
df = df[['Agency Name']]
print(df.value_counts())

Agency Name                                                                                
Department of Housing Preservation and Development                                             671543
Department of Transportation                                                                   333501
Department of Environmental Protection                                                         167866
New York City Police Department                                                                158664
Department of Buildings                                                                        129288
Department of Parks and Recreation                                                              66862
Department of Health and Mental Hygiene                                                         46725
Department of Consumer Affairs                                                                  27299
Taxi and Limousine Commission                                                               

In [108]:
df = improved_data_clean_agency(df, agency_abb)
print(df.value_counts())

Agency Name                                                                                
DEPARTMENT OF HOUSING PRESERVATION AND DEVELOPMENT                                             671543
DEPARTMENT OF TRANSPORTATION                                                                   333501
DEPARTMENT OF ENVIRONMENTAL PROTECTION                                                         167866
NEW YORK CITY POLICE DEPARTMENT                                                                158664
DEPARTMENT OF BUILDINGS                                                                        129288
DEPARTMENT OF PARKS AND RECREATION                                                              66862
DEPARTMENT OF HEALTH AND MENTAL HYGIENE                                                         46725
DEPARTMENT OF CONSUMER AFFAIRS                                                                  27299
TAXI AND LIMOUSINE COMMISSION                                                               

## data-cityofnewyork-us.acdt-2zt9.csv

In [109]:
datafile = data_dir + data_list[3]
df = dataset(datafile, encoding='utf-8')
df = df[['Agency Name']]
print(df.value_counts())

Agency Name                                                       
Department of Health and Mental Hygiene (DOHMH)                       22
Commission to Combat Police Corruption (CCPC)                         20
Department of Correction (DOC)                                        18
Mayor's Office of Immigrant Affairs (MOIA)                            14
Brooklyn Borough President (BPBK)                                     13
Department of Transportation (DOT)                                    11
Taxi and Limousine Commission (TLC)                                   11
Department of Citywide Administrative Services (DCAS)                 10
Department of City Planning (DCP)                                     10
Department of Buildings (DOB)                                         10
Department of Social Services (DSS) - Human Resources Agency (HRA)     9
Department of Parks and Recreation (DPR)                               8
Office of Management and Budget (OMB)                    

In [110]:
df = improved_data_clean_agency(df, agency_abb)
print(df.value_counts())

Agency Name                                                       
DEPARTMENT OF HEALTH AND MENTAL HYGIENE (DOHMH)                       22
COMMISSION TO COMBAT POLICE CORRUPTION (CCPC)                         20
DEPARTMENT OF CORRECTION (DOC)                                        18
MAYOR'S OFFICE OF IMMIGRANT AFFAIRS (MOIA)                            14
BROOKLYN BOROUGH PRESIDENT (BPBK)                                     13
DEPARTMENT OF TRANSPORTATION (DOT)                                    11
TAXI AND LIMOUSINE COMMISSION (TLC)                                   11
DEPARTMENT OF CITYWIDE ADMINISTRATIVE SERVICES (DCAS)                 10
DEPARTMENT OF CITY PLANNING (DCP)                                     10
DEPARTMENT OF BUILDINGS (DOB)                                         10
DEPARTMENT OF SOCIAL SERVICES (DSS) - HUMAN RESOURCES AGENCY (HRA)     9
DEPARTMENT OF PARKS AND RECREATION (DPR)                               8
OFFICE OF MANAGEMENT AND BUDGET (OMB)                    

## data-cityofnewyork-us.cwy2-px8b.csv

In [111]:
datafile = data_dir + data_list[4]
df = dataset(datafile, encoding='utf-8')
df = df[['Agency Name']]
print(df.value_counts())

Agency Name                    
New York City Police Department    14093
dtype: int64


In [112]:
df = improved_data_clean_agency(df, agency_abb)
print(df.value_counts())

Agency Name                    
NEW YORK CITY POLICE DEPARTMENT    14093
dtype: int64


## data-cityofnewyork-us.xrwg-eczf.csv

In [113]:
datafile = data_dir + data_list[5]
df = dataset(datafile, encoding='utf-8')
df = df[['Agency Name']]
print(df.value_counts())

Agency Name                                       
Human Resources Administration                        501
Department of Parks and Recreation                    301
Department of Buildings                               205
Department of Health and Mental Hygiene               192
Small Business Services                               143
Department of Education                               143
New York City Housing Authority                       112
Department of Transportation                           80
Department of Housing Preservation and Development     76
Department of Citywide Administrative Services         57
Office of Administrative Trials and Hearings           57
Department of Finance                                  56
New York City Police Department                        54
Department of City Planning                            51
Department of Environmental Protection                 47
Commission on Human Rights                             41
Taxi and Limousine Co

In [114]:
df = improved_data_clean_agency(df, agency_abb)
print(df.value_counts())

Agency Name                                       
HUMAN RESOURCES ADMINISTRATION                        501
DEPARTMENT OF PARKS AND RECREATION                    301
DEPARTMENT OF BUILDINGS                               205
DEPARTMENT OF HEALTH AND MENTAL HYGIENE               192
SMALL BUSINESS SERVICES                               143
DEPARTMENT OF EDUCATION                               143
NEW YORK CITY HOUSING AUTHORITY                       112
DEPARTMENT OF TRANSPORTATION                           80
DEPARTMENT OF HOUSING PRESERVATION AND DEVELOPMENT     76
DEPARTMENT OF CITYWIDE ADMINISTRATIVE SERVICES         57
OFFICE OF ADMINISTRATIVE TRIALS AND HEARINGS           57
DEPARTMENT OF FINANCE                                  56
NEW YORK CITY POLICE DEPARTMENT                        54
DEPARTMENT OF CITY PLANNING                            51
DEPARTMENT OF ENVIRONMENTAL PROTECTION                 47
COMMISSION ON HUMAN RIGHTS                             41
TAXI AND LIMOUSINE CO